<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F_%EC%83%81%EC%9E%A5%ED%9A%8C%EC%82%AC_%EB%B6%84%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dart-fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 3.0.1
    Uninstalling termcolor-3.0.1:
      Successfully uninstalled termcolor-3.0.1


# 분기보고서 다운로드 후 정리하기

In [7]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import re

# 1. API 엔드포인트 및 파라미터 설정 (DART Open API 사용)
API_KEY = "e5ebd99a0228714b70206b955cb7931ffc873783"  # 여기에 발급받은 API 키를 입력하세요
CORP_CODE = "00126380"  # 삼성전자 종목코드 예시, 실제 회사 코드로 변경
REPORT_CODE = "11011"  # 사업보고서 코드
QUARTER = "20234Q"  # 2023년 4분기

# 2. API 호출 및 XBRL 파일 다운로드
def download_xbrl(api_key, corp_code, report_code, bsns_year, reprt_code):
    """DART Open API를 통해 XBRL zip 파일을 다운로드합니다."""
    url = "https://opendart.fss.or.kr/api/list.xml"
    params = {
        "crtfc_key": api_key,
        "corp_code": corp_code,
        "bsns_year": bsns_year[:4],
        "reprt_code": reprt_code,
        "page_count": 100
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    xml_data = response.text

    # 필요한 정보 추출 (list.xml 응답에서)
    root = ET.fromstring(xml_data)
    ns = {'default': 'http://dart.fss.or.kr/dtd/ns/corpcode'}  # 네임스페이스 정의
    file_name = root.find(".//default:list/default:atch_file", namespaces=ns).text
    report_name = root.find(".//default:list/default:report_nm", namespaces=ns).text
    url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={api_key}&rcept_no={root.find('.//default:list/default:rcept_no', namespaces=ns).text}"
    # URL 및 API 키를 사용하여 document.xml 다운로드 시도
    response = requests.get(url)
    response.raise_for_status()
    xml_data = response.text
    root = ET.fromstring(xml_data)
    link = root.find('.//default:list/default:atch_file', namespaces=ns).text
    link_str = f"https://opendart.fss.or.kr/api/download.zip?crtfc_key={api_key}&atch_file={root.find('.//default:list/default:atch_file', namespaces=ns).text}&rcept_no={root.find('.//default:list/default:rcept_no', namespaces=ns).text}"
    # 다운로드 URL
    download_url = f"https://opendart.fss.or.kr/api/download.zip?crtfc_key={api_key}&atch_file={link}&rcept_no={root.find('.//default:list/default:rcept_no', namespaces=ns).text}"


    zip_response = requests.get(download_url)
    zip_response.raise_for_status()  # 에러 발생 시 예외 처리
    return io.BytesIO(zip_response.content)

# 3. XBRL 파일 파싱 및 계정과목 추출
def parse_xbrl(zip_file):
    """XBRL zip 파일에서 계정과목 및 값을 추출합니다."""
    account_data = []
    with zipfile.ZipFile(zip_file, "r") as zf:
        for filename in zf.namelist():
            if filename.endswith(".xml"):  # XBRL 파일만 처리
                try:
                    with zf.open(filename) as xml_file:
                        tree = ET.parse(xml_file)
                        root = tree.getroot()

                        # 네임스페이스 처리 (네임스페이스가 있는 경우)
                        ns = {'xbrli': 'http://www.xbrl.org/2003/instance'}
                        # 필요한 정보 추출 (계정과목 이름, 값 등)
                        for element in root.findall(".//*[@contextRef]", namespaces=ns):
                            context_ref = element.get('contextRef')
                            # 각 element.tag 에서 account name 추출
                            tag_name = element.tag.split("}")[-1]
                            # 값 추출
                            text_value = element.text
                            # 계정과목명, 값을 튜플로 저장
                            account_data.append((tag_name, text_value, context_ref))

                except Exception as e:
                    print(f"Error parsing {filename}: {e}")
    return account_data

# 4. 데이터 프레임 생성 및 결과 확인
def create_dataframe(account_data):
    """추출된 계정 데이터로 Pandas DataFrame을 생성합니다."""
    df = pd.DataFrame(account_data, columns=["계정과목", "값", "ContextRef"])
    return df

# 5. Main 실행 함수
if __name__ == "__main__":
    try:
        # XBRL 파일 다운로드
        zip_file = download_xbrl(API_KEY, CORP_CODE, REPORT_CODE, QUARTER, "11011")

        # XBRL 파일 파싱
        account_data = parse_xbrl(zip_file)

        # 데이터 프레임 생성
        df = create_dataframe(account_data)

        # 결과 출력 (예시)
        print(df.head())
        # 필요하다면 CSV 파일로 저장
        # df.to_csv("account_data.csv", encoding="utf-8-sig", index=False)

    except Exception as e:
        print(f"오류 발생: {e}")

오류 발생: 'NoneType' object has no attribute 'text'


In [6]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = 'e5ebd99a0228714b70206b955cb7931ffc873783'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L1", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': "20234Q",  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": "20234Q",
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_1 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L1.xlsx"

df_1.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

❌ (주)파라다이스(00171265) - API 오류: 조회된 데이타가 없습니다.
❌ 한국타이어앤테크놀로지(주)(00937324) - API 오류: 조회된 데이타가 없습니다.
❌ 한국철강 주식회사(00687711) - API 오류: 조회된 데이타가 없습니다.
❌ 대원강업 주식회사(00111847) - API 오류: 조회된 데이타가 없습니다.
❌ 키스코홀딩스 주식회사(00159740) - API 오류: 조회된 데이타가 없습니다.
❌ (주)팜젠사이언스(00135050) - API 오류: 조회된 데이타가 없습니다.
❌ (주)지니뮤직(00223762) - API 오류: 조회된 데이타가 없습니다.
❌ 새론오토모티브(주)(00305668) - API 오류: 조회된 데이타가 없습니다.


KeyboardInterrupt: 

#
:------------------------------------------------------------------------------L2------------------------------------

In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L2", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_2 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L2.xlsx"

df_2.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

✅ (주)휴비츠(00398668) - 데이터 수집 완료
✅ 주식회사 휴스틸(00156488) - 데이터 수집 완료
✅ 주식회사 엔지켐생명과학(00606886) - 데이터 수집 완료
✅ 삼화전자공업(주)(00129280) - 데이터 수집 완료
✅ 주식회사 덕신이피씨(00454937) - 데이터 수집 완료
✅ 엔에이치엔(주)(00983271) - 데이터 수집 완료
✅ 피에이치에이 주식회사(00155151) - 데이터 수집 완료
✅ (주)에이스테크놀로지(00556907) - 데이터 수집 완료
✅ 아이쓰리시스템(주)(00612489) - 데이터 수집 완료
✅ 대한약품공업주식회사(00113128) - 데이터 수집 완료
✅ (주)서한(00131504) - 데이터 수집 완료
✅ 주식회사 라닉스(01327092) - 데이터 수집 완료
✅ 스튜디오드래곤(주)(01168684) - 데이터 수집 완료
✅ (주)앤디포스(00942131) - 데이터 수집 완료
❌ 유성티엔에스(00173999) - API 오류: 조회된 데이타가 없습니다.
✅ (주)까스텔바작(01181807) - 데이터 수집 완료
✅ 주식회사 세토피아(01091382) - 데이터 수집 완료
✅ 광전자(주)(00152729) - 데이터 수집 완료
✅ (주)골드앤에스(00249502) - 데이터 수집 완료
✅ (주)인포바인(00491415) - 데이터 수집 완료
✅ 주식회사 남해화학(00107987) - 데이터 수집 완료
✅ 하이트진로홀딩스 주식회사(00148993) - 데이터 수집 완료
✅ (주)알에프텍(00309831) - 데이터 수집 완료
✅ 대원제약 주식회사(00111999) - 데이터 수집 완료
✅ 패션플랫폼(주)(01101041) - 데이터 수집 완료
✅ (주)씨유메디칼시스템(00681249) - 데이터 수집 완료
✅ 로체시스템즈 주식회사(00363486) - 데이터 수집 완료
✅ (주)이수앱지스(00526872) - 데이터 수집 완료
✅ 주식회사 휴센텍(01063884) - 데이터 수집 완료
✅ (주)한솔케미

#
:------------------------------------------------------------------------------L3------------------------------------




In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L3", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_3 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L3.xlsx"

df_3.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

✅ 포스코스틸리온(주)(00155258) - 데이터 수집 완료
✅ 비플라이소프트(주)(00980043) - 데이터 수집 완료
✅ 씨유테크주식회사(00575106) - 데이터 수집 완료
✅ 송원산업 주식회사(00134963) - 데이터 수집 완료
✅ KG스틸 주식회사(00115676) - 데이터 수집 완료
✅ (주)씨티프라퍼티(00349811) - 데이터 수집 완료
✅ 주식회사 셀바스헬스케어(01047451) - 데이터 수집 완료
✅ 주식회사 셀바스에이아이(00666064) - 데이터 수집 완료
✅ 주식회사 디지피(00367695) - 데이터 수집 완료
✅ 주식회사 앱코(01207716) - 데이터 수집 완료
✅ 주식회사 툴젠(00547510) - 데이터 수집 완료
❌ 주식회사 디에스앤엘(00812362) - API 오류: 조회된 데이타가 없습니다.
✅ 주식회사 디에스케이(00530121) - 데이터 수집 완료
✅ 메가스터디 주식회사(00422284) - 데이터 수집 완료
✅ 코닉오토메이션(주)(01573284) - 데이터 수집 완료
✅ 주식회사 알비더블유(01303029) - 데이터 수집 완료
✅ 한국내화주식회사(00160205) - 데이터 수집 완료
✅ 주식회사 한네트(00244783) - 데이터 수집 완료
✅ 주식회사 씨아이테크(00127158) - 데이터 수집 완료
✅ 풍원정밀(주)(00417167) - 데이터 수집 완료
✅ (주)골프존뉴딘홀딩스(00674498) - 데이터 수집 완료
✅ (주)태웅로직스(00562245) - 데이터 수집 완료
✅ (주)한송네오텍(01101722) - 데이터 수집 완료
✅ (주)모토닉(00151128) - 데이터 수집 완료
✅ (주)마이크로컨텍솔루션(00568188) - 데이터 수집 완료
✅ (주)에스피시스템스(00624749) - 데이터 수집 완료
✅ 주식회사 엘앤케이바이오메드(00918222) - 데이터 수집 완료
✅ 주식회사 뷰노(01344202) - 데이터 수집 완료
✅ 금강철강 주식회사(00105475) - 데이터

#
:------------------------------------------------------------------------------L4------------------------------------

In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L4", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_4 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L4.xlsx"

df_4.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

✅ 에이치케이이노엔 주식회사(01012066) - 데이터 수집 완료
✅ 한국단자공업(주)(00157070) - 데이터 수집 완료
✅ 주식회사 서부티엔디(00130383) - 데이터 수집 완료
✅ 우리바이오 주식회사(00369107) - 데이터 수집 완료
✅ 우리엔터프라이즈 주식회사(00155692) - 데이터 수집 완료
✅ 씨제이프레시웨이(주)(00127954) - 데이터 수집 완료
✅ 주식회사 코콤(00187725) - 데이터 수집 완료
✅ 삼성제약 주식회사(00126414) - 데이터 수집 완료
✅ 이지트로닉스(01140837) - 데이터 수집 완료
✅ 대보마그네틱 주식회사(01259418) - 데이터 수집 완료
✅ 주식회사 동성케미컬(00679314) - 데이터 수집 완료
✅ 주식회사 웹스(00864338) - 데이터 수집 완료
✅ 주식회사 삼영에스앤씨(00405506) - 데이터 수집 완료
✅ 디와이씨 주식회사(01336443) - 데이터 수집 완료
✅ 주식회사 홈센타홀딩스(00288495) - 데이터 수집 완료
✅ 지엘팜텍 주식회사(01042359) - 데이터 수집 완료
✅ 주식회사 비스토스(01506848) - 데이터 수집 완료
✅ (주)동운아나텍(00695969) - 데이터 수집 완료
✅ (주)티씨씨스틸(00117300) - 데이터 수집 완료
✅ 주식회사 토탈소프트뱅크(00355089) - 데이터 수집 완료
✅ 신라교역 주식회사(00135962) - 데이터 수집 완료
✅ 고려신용정보(00336297) - 데이터 수집 완료
✅ (주)모다이노칩(00480048) - 데이터 수집 완료
✅ (주)에스엠라이프디자인그룹(00367604) - 데이터 수집 완료
✅ 바이젠셀(주)(01335790) - 데이터 수집 완료
✅ 무림페이퍼 주식회사(00136086) - 데이터 수집 완료
✅ (주)다우데이타(00238782) - 데이터 수집 완료
✅ 주식회사 아스트(00409681) - 데이터 수집 완료
✅ 주식회사 세보엠이씨(00133876) - 데이터 수집 완료
✅ 

#
:------------------------------------------------------------------------------L5------------------------------------

In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L5", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_5 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L5.xlsx"

df_5.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

✅ 주식회사 디티앤씨(00675594) - 데이터 수집 완료
✅ 덕산하이메탈 주식회사(00471068) - 데이터 수집 완료
✅ 주식회사 현대리바트(00300548) - 데이터 수집 완료
✅ KPX케미칼 주식회사(00191472) - 데이터 수집 완료
✅ 에어부산 주식회사(00651901) - 데이터 수집 완료
✅ KPX홀딩스 주식회사(00587466) - 데이터 수집 완료
✅ 케이에이치필룩스 주식회사(00193009) - 데이터 수집 완료
✅ 주식회사 에코바이브(00108065) - 데이터 수집 완료
✅ (주)비츠로시스(00201432) - 데이터 수집 완료
✅ (주)SG세계물산(00133511) - 데이터 수집 완료
✅ (주)와이지엔터테인먼트(00613318) - 데이터 수집 완료
✅ 세종공업주식회사(00134510) - 데이터 수집 완료
✅ 디씨엠 주식회사(00177199) - 데이터 수집 완료
✅ AK홀딩스(주)(00125080) - 데이터 수집 완료
✅ (주)서진오토모티브(00800145) - 데이터 수집 완료
✅ 옴니시스템 주식회사(00265005) - 데이터 수집 완료
✅ 주식회사 메디톡스(00580199) - 데이터 수집 완료
✅ 영화금속 주식회사(00141413) - 데이터 수집 완료
✅ 가온그룹㈜(00441304) - 데이터 수집 완료
✅ (주)수산중공업(00135111) - 데이터 수집 완료
✅ (주)에프엔씨엔터테인먼트(00925295) - 데이터 수집 완료
✅ 주식회사 케이엠(00242934) - 데이터 수집 완료
✅ 주식회사 케이지모빌리언스(00405278) - 데이터 수집 완료
✅ 부광약품(주)(00123718) - 데이터 수집 완료
✅ 사조산업 주식회사(00124799) - 데이터 수집 완료
✅ 태양금속공업 주식회사(00153755) - 데이터 수집 완료
✅ 주식회사 이랜텍(00206659) - 데이터 수집 완료
✅ ㈜예스티(00530556) - 데이터 수집 완료
✅ (주)키이스트(00353762) - 데이터 수집 완료
✅ 조광페인트

In [ ]:
df_fn = pd.concat([df_1,df_2,df_3,df_4,df_5])
df_fn.to_excel('상장법인재무제표_중복제거.xlsx')